# CPU Rust vs Python Cosine Search Suite

Ten randomized synthetic scenarios compare Rust-backed `NNdex` (`approx=False`)
against a pure NumPy cosine top-k baseline.

Primary metric: `rust_to_python_speed_ratio` where values `> 1` mean Rust is faster.


In [1]:
from dataclasses import dataclass
from time import perf_counter
from typing import Callable

import numpy as np
import pandas as pd
from nndex import NNdex

EPSILON = 1e-12

MatrixBuilder = Callable[[np.random.Generator, int, int], np.ndarray]


@dataclass(frozen=True)
class Scenario:
    name: str
    rows: int
    dims: int
    query_rows: int
    k: int
    tolerance: float
    builder: MatrixBuilder


def normalize_rows(matrix: np.ndarray) -> np.ndarray:
    norms = np.linalg.norm(matrix, axis=1, keepdims=True).astype(np.float32)
    return (matrix / np.maximum(norms, np.float32(EPSILON))).astype(np.float32, copy=False)


def python_cosine_topk_batch(
    normalized_matrix: np.ndarray,
    queries: np.ndarray,
    k: int,
) -> tuple[np.ndarray, np.ndarray]:
    q_norm = normalize_rows(queries)
    sims = q_norm @ normalized_matrix.T

    candidate_idx = np.argpartition(sims, -k, axis=1)[:, -k:]
    row_ids = np.arange(sims.shape[0])[:, None]
    candidate_scores = sims[row_ids, candidate_idx]
    order = np.argsort(candidate_scores, axis=1)[:, ::-1]

    topk_idx = candidate_idx[row_ids, order].astype(np.int64, copy=False)
    topk_scores = candidate_scores[row_ids, order].astype(np.float32, copy=False)
    return topk_idx, topk_scores


def python_cosine_topk_single(
    normalized_matrix: np.ndarray,
    query: np.ndarray,
    k: int,
) -> tuple[np.ndarray, np.ndarray]:
    """Single-query cosine top-k via matrix-vector product (BLAS sgemv path)."""
    norm = np.float32(max(float(np.linalg.norm(query)), EPSILON))
    q_norm = (query / norm).astype(np.float32, copy=False)
    sims = normalized_matrix @ q_norm

    candidate_idx = np.argpartition(sims, -k)[-k:]
    candidate_scores = sims[candidate_idx]
    order = np.argsort(candidate_scores)[::-1]

    topk_idx = candidate_idx[order].astype(np.int64, copy=False)
    topk_scores = candidate_scores[order].astype(np.float32, copy=False)
    return topk_idx, topk_scores


def topk_overlap_fraction(rust_idx: np.ndarray, python_idx: np.ndarray) -> float:
    overlaps: list[float] = []
    for rust_row, py_row in zip(rust_idx, python_idx):
        overlaps.append(len(set(rust_row.tolist()) & set(py_row.tolist())) / rust_row.size)
    return float(np.mean(overlaps))


def median_ms(func: Callable[[], object], repeats: int) -> float:
    timings = np.empty(repeats, dtype=np.float64)
    for i in range(repeats):
        start = perf_counter()
        _ = func()
        timings[i] = (perf_counter() - start) * 1_000.0
    return float(np.median(timings))


def build_queries(matrix: np.ndarray, rng: np.random.Generator, query_rows: int) -> np.ndarray:
    row_ids = rng.integers(0, matrix.shape[0], size=query_rows)
    jitter = rng.normal(loc=0.0, scale=0.02, size=(query_rows, matrix.shape[1]))
    return (matrix[row_ids] + jitter.astype(np.float32)).astype(np.float32, copy=False)


# Unusual: multiplicative/nonlinear features that stress interaction sensitivity.
def interaction_effects_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    core = rng.normal(size=(rows, 8)).astype(np.float32)
    interactions = np.column_stack(
        [
            core[:, 0] * core[:, 1],
            core[:, 0] * core[:, 2],
            core[:, 1] * core[:, 3],
            core[:, 4] * core[:, 5],
            np.sin(core[:, 6]),
            np.cos(core[:, 7]),
            core[:, 2] ** 2,
            core[:, 3] ** 2,
        ]
    ).astype(np.float32)
    features = np.concatenate([core, interactions], axis=1)
    if features.shape[1] >= dims:
        return features[:, :dims].astype(np.float32, copy=False)
    pad = rng.normal(0.0, 0.05, size=(rows, dims - features.shape[1])).astype(np.float32)
    return np.concatenate([features, pad], axis=1)


# Unusual: vectors constrained to noisy radius shells; tests angular ranking robustness.
def concentric_shells_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    directions = rng.normal(size=(rows, dims)).astype(np.float32)
    norms = np.linalg.norm(directions, axis=1, keepdims=True).astype(np.float32)
    radii = rng.choice(np.array([0.55, 1.25, 2.1, 3.4], dtype=np.float32), size=rows)
    noise = rng.normal(loc=0.0, scale=0.03, size=(rows, dims)).astype(np.float32)
    shells = directions / np.maximum(norms, np.float32(EPSILON))
    return (shells * radii[:, None] + noise).astype(np.float32, copy=False)


# Unusual: low-rank signal plus noise; tests behavior on compressible embeddings.
def low_rank_plus_noise_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    rank = min(12, max(4, dims // 8))
    latent = rng.normal(size=(rows, rank)).astype(np.float32)
    loadings = rng.normal(size=(rank, dims)).astype(np.float32)
    structured = latent @ loadings
    noise = rng.normal(loc=0.0, scale=0.04, size=(rows, dims)).astype(np.float32)
    return (structured + noise).astype(np.float32, copy=False)


# Unusual: mostly sparse spikes with periodic drift; tests sparse/high-dim retrieval.
def sparse_spikes_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    matrix = rng.normal(loc=0.0, scale=0.015, size=(rows, dims)).astype(np.float32)
    spikes = max(2, dims // 32)
    row_ids = np.repeat(np.arange(rows), spikes)
    col_ids = rng.integers(0, dims, size=rows * spikes)
    values = rng.normal(loc=0.0, scale=1.2, size=rows * spikes).astype(np.float32)
    matrix[row_ids, col_ids] += values
    matrix += np.sin(np.arange(dims, dtype=np.float32) / 9.0)[None, :] * np.float32(0.08)
    return matrix


# Unusual: curved manifold projected into feature space; tests non-Euclidean structure.
def curve_manifold_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    t = rng.uniform(0.0, 6.0 * np.pi, size=(rows, 1)).astype(np.float32)
    manifold = np.concatenate(
        [np.sin(t), np.cos(t), np.sin(2.0 * t), np.cos(2.0 * t), t / np.float32(6.0 * np.pi)],
        axis=1,
    ).astype(np.float32)
    projection = rng.normal(size=(manifold.shape[1], dims)).astype(np.float32)
    embedded = manifold @ projection
    noise = rng.normal(loc=0.0, scale=0.025, size=(rows, dims)).astype(np.float32)
    return (embedded + noise).astype(np.float32, copy=False)


# Unusual: block-correlated dimensions; tests ranking under covariance clusters.
def block_correlated_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    blocks = max(3, min(12, dims // 8))
    block_width = (dims + blocks - 1) // blocks
    latent = rng.normal(size=(rows, blocks)).astype(np.float32)
    matrix = np.empty((rows, dims), dtype=np.float32)
    for block_idx in range(blocks):
        start = block_idx * block_width
        end = min(dims, start + block_width)
        if start >= end:
            break
        jitter = rng.normal(loc=0.0, scale=0.06, size=(rows, end - start)).astype(np.float32)
        matrix[:, start:end] = latent[:, [block_idx]] + jitter
    return matrix


# Unusual: chirped sinusoid feature banks; tests structured periodic variation.
def chirp_wave_bank_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    x = np.linspace(0.0, 1.0, num=dims, dtype=np.float32)[None, :]
    base_freq = rng.uniform(1.5, 16.0, size=(rows, 1)).astype(np.float32)
    chirp = rng.uniform(0.5, 5.0, size=(rows, 1)).astype(np.float32)
    phase = rng.uniform(0.0, 2.0 * np.pi, size=(rows, 1)).astype(np.float32)
    waves = np.sin((base_freq * x + chirp * x * x) * np.float32(2.0 * np.pi) + phase)
    harmonics = np.cos((base_freq * np.float32(0.5) * x) * np.float32(2.0 * np.pi) - phase)
    noise = rng.normal(loc=0.0, scale=0.03, size=(rows, dims)).astype(np.float32)
    return (waves + np.float32(0.35) * harmonics + noise).astype(np.float32, copy=False)


# Unusual: piecewise trends with jumps; tests retrieval on nonstationary vectors.
def piecewise_jump_trend_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    segments = 5
    boundaries = np.linspace(0, dims, num=segments + 1, dtype=np.int64)
    matrix = rng.normal(loc=0.0, scale=0.08, size=(rows, dims)).astype(np.float32)
    slopes = rng.normal(loc=0.0, scale=0.9, size=(rows, segments)).astype(np.float32)
    for segment_idx in range(segments):
        start = int(boundaries[segment_idx])
        end = int(boundaries[segment_idx + 1])
        if start >= end:
            continue
        ramp = np.linspace(0.0, 1.0, num=end - start, dtype=np.float32)[None, :]
        matrix[:, start:end] += slopes[:, [segment_idx]] * ramp
        if segment_idx + 1 < segments:
            jump = rng.normal(loc=0.0, scale=0.35, size=(rows, 1)).astype(np.float32)
            matrix[:, end:] += jump
    seasonal = np.sin(np.arange(dims, dtype=np.float32) / np.float32(7.0))[None, :]
    return (matrix + np.float32(0.05) * seasonal).astype(np.float32, copy=False)


# Unusual: mixed positive/negative topic prototypes; tests signed semantic superposition.
def signed_topic_mixture_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    topic_count = 20
    topics = rng.normal(size=(topic_count, dims)).astype(np.float32)
    topic_ids_a = rng.integers(0, topic_count, size=rows)
    topic_ids_b = rng.integers(0, topic_count, size=rows)
    blend = rng.uniform(-1.0, 1.0, size=(rows, 1)).astype(np.float32)
    matrix = topics[topic_ids_a] * blend + topics[topic_ids_b] * (np.float32(1.0) - blend)
    gates = rng.choice(np.array([-1.0, 1.0], dtype=np.float32), size=(rows, 1))
    noise = rng.normal(loc=0.0, scale=0.02, size=(rows, dims)).astype(np.float32)
    return (matrix * gates + noise).astype(np.float32, copy=False)


# Unusual: binary hashes projected to dense space; tests discontinuous-to-dense embeddings.
def binary_hash_projection_matrix(rng: np.random.Generator, rows: int, dims: int) -> np.ndarray:
    bit_count = 64
    bits = rng.integers(0, 2, size=(rows, bit_count), dtype=np.int8).astype(np.float32)
    signed_bits = bits * np.float32(2.0) - np.float32(1.0)
    projection = rng.normal(loc=0.0, scale=0.2, size=(bit_count, dims)).astype(np.float32)
    dense = signed_bits @ projection
    trend = rng.normal(loc=0.0, scale=0.04, size=(rows, 1)).astype(np.float32)
    axis = np.linspace(-1.0, 1.0, num=dims, dtype=np.float32)[None, :]
    return (dense + trend * axis).astype(np.float32, copy=False)


BATCH_SCENARIOS = (
    Scenario('interaction_effects', 12_345, 33, 128, 15, 1e-4, interaction_effects_matrix),
    Scenario('concentric_shells', 9_876, 129, 128, 15, 1e-4, concentric_shells_matrix),
    Scenario('low_rank_plus_noise', 15_555, 192, 128, 15, 1e-4, low_rank_plus_noise_matrix),
    Scenario('sparse_spikes', 18_181, 640, 96, 15, 2e-4, sparse_spikes_matrix),
    Scenario('curve_manifold_projection', 7_777, 11, 128, 15, 1e-4, curve_manifold_matrix),
    Scenario('block_correlated_large', 50_000, 48, 160, 20, 1e-4, block_correlated_matrix),
    Scenario('chirp_wave_bank_large', 55_555, 96, 160, 20, 1e-4, chirp_wave_bank_matrix),
    Scenario('piecewise_jump_trend_large', 62_500, 257, 128, 20, 1e-4, piecewise_jump_trend_matrix),
    Scenario('signed_topic_mixture_large', 70_000, 384, 96, 20, 2e-4, signed_topic_mixture_matrix),
    Scenario('binary_hash_projection_large', 80_000, 768, 64, 20, 2e-4, binary_hash_projection_matrix),
)

SINGLE_QUERY_SCENARIOS = (
    Scenario('interaction_effects', 12_345, 33, 1, 15, 1e-4, interaction_effects_matrix),
    Scenario('concentric_shells', 9_876, 129, 1, 15, 1e-4, concentric_shells_matrix),
    Scenario('low_rank_plus_noise', 15_555, 192, 1, 15, 1e-4, low_rank_plus_noise_matrix),
    Scenario('sparse_spikes', 18_181, 640, 1, 15, 2e-4, sparse_spikes_matrix),
    Scenario('curve_manifold_projection', 7_777, 11, 1, 15, 1e-4, curve_manifold_matrix),
    Scenario('block_correlated_large', 50_000, 48, 1, 20, 1e-4, block_correlated_matrix),
    Scenario('chirp_wave_bank_large', 55_555, 96, 1, 20, 1e-4, chirp_wave_bank_matrix),
    Scenario('piecewise_jump_trend_large', 62_500, 257, 1, 20, 1e-4, piecewise_jump_trend_matrix),
    Scenario('signed_topic_mixture_large', 70_000, 384, 1, 20, 2e-4, signed_topic_mixture_matrix),
    Scenario('binary_hash_projection_large', 80_000, 768, 1, 20, 2e-4, binary_hash_projection_matrix),
)


def run_suite(repeats: int = 7) -> pd.DataFrame:
    rng = np.random.default_rng()

    rows: list[dict[str, float | int | str]] = []
    for s in BATCH_SCENARIOS:
        matrix = s.builder(rng, s.rows, s.dims)
        queries = build_queries(matrix, rng, s.query_rows)

        rust_index = NNdex(
            matrix,
            normalized=False,
            approx=False,
            backend='cpu',
            enable_cache=False,
        )

        normalized_matrix = normalize_rows(matrix)
        rust_idx, rust_scores = rust_index.search(queries, k=s.k)
        py_idx, py_scores = python_cosine_topk_batch(normalized_matrix, queries, s.k)

        overlap = topk_overlap_fraction(rust_idx, py_idx)
        max_delta = float(np.max(np.abs(rust_scores - py_scores)))
        assert overlap >= 0.995, f'{s.name}: top-k overlap {overlap:.6f} < 0.995'
        assert max_delta <= s.tolerance, (
            f'{s.name}: max similarity delta {max_delta:.6g} > tolerance {s.tolerance:.6g}'
        )

        rust_ms = median_ms(lambda: rust_index.search(queries, k=s.k), repeats)
        py_ms = median_ms(
            lambda: python_cosine_topk_batch(normalized_matrix, queries, s.k),
            repeats,
        )
        rows.append(
            {
                'scenario': s.name,
                'rows': s.rows,
                'dims': s.dims,
                'query_rows': s.query_rows,
                'k': s.k,
                'rust_median_ms': rust_ms,
                'python_median_ms': py_ms,
                'rust_to_python_speed_ratio': py_ms / rust_ms,
                'topk_overlap': overlap,
                'max_similarity_abs_delta': max_delta,
                'similarity_tolerance': s.tolerance,
            }
        )

    return pd.DataFrame(rows).sort_values('rust_to_python_speed_ratio', ascending=False)


def run_single_query_suite(repeats: int = 31) -> pd.DataFrame:
    """Same scenarios as batch suite but with query_rows=1 (single-query code path).

    Uses a bank of 32 queries and cycles through them across repeats to reduce
    bias from a single query's cache behavior. Median is taken over all timings.
    """
    rng = np.random.default_rng()
    query_bank_size = 32

    rows: list[dict[str, float | int | str]] = []
    for s in SINGLE_QUERY_SCENARIOS:
        matrix = s.builder(rng, s.rows, s.dims)
        query_bank = build_queries(matrix, rng, query_bank_size)

        rust_index = NNdex(
            matrix,
            normalized=False,
            approx=False,
            backend='cpu',
            enable_cache=False,
        )

        normalized_matrix = normalize_rows(matrix)

        # Correctness check on each query in the bank
        max_delta = np.float32(0.0)
        min_overlap = 1.0
        for qi in range(query_bank_size):
            query = query_bank[qi]
            rust_idx, rust_scores = rust_index.search(query, k=s.k)
            py_idx, py_scores = python_cosine_topk_single(normalized_matrix, query, s.k)
            overlap = len(set(rust_idx.tolist()) & set(py_idx.tolist())) / len(rust_idx)
            delta = float(np.max(np.abs(rust_scores - py_scores)))
            min_overlap = min(min_overlap, overlap)
            max_delta = max(max_delta, delta)

        assert min_overlap >= 0.995, (
            f'{s.name}: single-query top-k overlap {min_overlap:.6f} < 0.995'
        )
        assert max_delta <= s.tolerance, (
            f'{s.name}: single-query max similarity delta {max_delta:.6g} '
            f'> tolerance {s.tolerance:.6g}'
        )

        # Benchmark: cycle through query bank for stable timing
        cycle_idx = 0

        def rust_single() -> object:
            nonlocal cycle_idx
            q = query_bank[cycle_idx % query_bank_size]
            cycle_idx += 1
            return rust_index.search(q, k=s.k)

        cycle_idx_py = 0

        def python_single() -> object:
            nonlocal cycle_idx_py
            q = query_bank[cycle_idx_py % query_bank_size]
            cycle_idx_py += 1
            return python_cosine_topk_single(normalized_matrix, q, s.k)

        rust_ms = median_ms(rust_single, repeats)
        py_ms = median_ms(python_single, repeats)

        rows.append(
            {
                'scenario': s.name,
                'rows': s.rows,
                'dims': s.dims,
                'query_rows': 1,
                'k': s.k,
                'rust_median_ms': rust_ms,
                'python_median_ms': py_ms,
                'rust_to_python_speed_ratio': py_ms / rust_ms,
                'topk_overlap': min_overlap,
                'max_similarity_abs_delta': float(max_delta),
                'similarity_tolerance': s.tolerance,
            }
        )

    return pd.DataFrame(rows).sort_values('rust_to_python_speed_ratio', ascending=False)

## Batch Search (query_rows > 1)

Each scenario sends a batch of queries through `NNdex.search()` (2D input),
which routes to the Rust `search_batch` path. High-dimensional scenarios
(dims >= 64) use BLAS-backed GEMM for the dot-product computation.


In [2]:
results_df = run_suite(repeats=7)
results_df.round(
    {
        'rust_median_ms': 3,
        'python_median_ms': 3,
        'rust_to_python_speed_ratio': 3,
        'topk_overlap': 3,
        'max_similarity_abs_delta': 8,
        'similarity_tolerance': 8,
    }
).reset_index(drop=True)


scenario   rows  dims  query_rows   k  rust_median_ms  \
0     curve_manifold_projection   7777    11         128  15           0.797   
1           interaction_effects  12345    33         128  15           1.952   
2        block_correlated_large  50000    48         160  20           9.811   
3         chirp_wave_bank_large  55555    96         160  20          18.953   
4             concentric_shells   9876   129         128  15           3.876   
5           low_rank_plus_noise  15555   192         128  15           7.972   
6    piecewise_jump_trend_large  62500   257         128  20          43.642   
7    signed_topic_mixture_large  70000   384          96  20          57.830   
8                 sparse_spikes  18181   640          96  15          24.359   
9  binary_hash_projection_large  80000   768          64  20          91.001   

   python_median_ms  rust_to_python_speed_ratio  topk_overlap  \
0             7.195                       9.024         1.000   
1            11.084                       5.679         1.000   
2            53.062                       5.408         1.000   
3            62.464                       3.296         1.000   
4             8.953                       2.310         1.000   
5            13.658                       1.713         1.000   
6            57.111                       1.309         1.000   
7            47.872                       0.828         0.998   
8            13.501                       0.554         1.000   
9            43.173                       0.474         1.000   

   max_similarity_abs_delta  similarity_tolerance  
0              3.000000e-07                0.0001  
1              3.000000e-07                0.0001  
2              4.200000e-07                0.0001  
3              4.800000e-07                0.0001  
4              3.000000e-07                0.0001  
5              4.200000e-07                0.0001  
6              4.800000e-07                0.0001  
7              8.300000e-07                0.0002  
8              8.900000e-07                0.0002  
9              1.130000e-06                0.0002

## Single Query Search (query_rows = 1)

Same ten scenarios but with a single 1D query vector per call, exercising the
Rust `search` (not `search_batch`) code path. The Python baseline uses a
matrix-vector product (`normalized_matrix @ q_norm`) rather than a matrix-matrix
GEMM, so this measures a fundamentally different workload.


In [3]:
single_query_df = run_single_query_suite(repeats=31)
single_query_df.round(
    {
        'rust_median_ms': 3,
        'python_median_ms': 3,
        'rust_to_python_speed_ratio': 3,
        'topk_overlap': 3,
        'max_similarity_abs_delta': 8,
        'similarity_tolerance': 8,
    }
).reset_index(drop=True)

scenario   rows  dims  query_rows   k  rust_median_ms  \
0        block_correlated_large  50000    48           1  20           0.126   
1         chirp_wave_bank_large  55555    96           1  20           0.210   
2           interaction_effects  12345    33           1  15           0.052   
3     curve_manifold_projection   7777    11           1  15           0.029   
4    piecewise_jump_trend_large  62500   257           1  20           0.730   
5             concentric_shells   9876   129           1  15           0.078   
6    signed_topic_mixture_large  70000   384           1  20           1.079   
7           low_rank_plus_noise  15555   192           1  15           0.125   
8  binary_hash_projection_large  80000   768           1  20           2.294   
9                 sparse_spikes  18181   640           1  15           0.493   

   python_median_ms  rust_to_python_speed_ratio  topk_overlap  \
0             0.613                       4.876           1.0   
1             1.012                       4.824           1.0   
2             0.162                       3.106           1.0   
3             0.081                       2.804           1.0   
4             1.240                       1.698           1.0   
5             0.130                       1.663           1.0   
6             1.643                       1.523           1.0   
7             0.185                       1.474           1.0   
8             2.896                       1.262           1.0   
9             0.567                       1.149           1.0   

   max_similarity_abs_delta  similarity_tolerance  
0              3.000000e-07                0.0001  
1              3.000000e-07                0.0001  
2              3.600000e-07                0.0001  
3              2.400000e-07                0.0001  
4              5.400000e-07                0.0001  
5              3.000000e-07                0.0001  
6              4.800000e-07                0.0002  
7              4.200000e-07                0.0001  
8              4.200000e-07                0.0002  
9              8.300000e-07                0.0002